In [166]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [167]:
url = "http://aligulac.com/results/"

driver = webdriver.Chrome()
driver.implicitly_wait(5)
driver.get(url)

In [168]:
NumberOfPages = 1000

PlayerAName = []
PlayerBName = []
PlayerARating = []
PlayerBRating = []
PlayerACountry = []
PlayerBCountry = []
PlayerARace = []
PlayerBRace = []
MapScore = []
isRatedMatch = []
GameVersion = []
isLan = []
PlayerAWin = []

#Number of Pages to scrap from
for i in range(0, NumberOfPages):
    soup = BeautifulSoup(driver.page_source, "html.parser")
    #Loop through all table bodies on the page
    for tbody in soup('tbody'):
        #Loop through each table row in the table body
        for tr in tbody:
            #Loop through each table cell in te table row
            for td in tr:
                #Skip empty whitespce only cell
                if(len(str(td)) > 1):
                    try:
                        #Player A
                        if(td["class"][0] == "lm_rta"):
                            PlayerARating.append(str(td.contents[0]).strip())
                        elif(td["class"][0] == "lm_pla"):
                            #Some player information cells are missing the players country. len(td) of 7 indicates all fields present and 5 as missing the country info
                            if(len(td) == 7):
                                idx = 0
                                for e in td:
                                    #Ignore whitespace elements
                                    if(len(str(e)) > 1):
                                        #Player A's nmae appears first
                                        if(idx == 0):
                                            PlayerAName.append(str(e.contents[0]).strip())
                                            idx += 1
                                        #Players A's country appears second
                                        elif(idx == 1):
                                            PlayerACountry.append(str(e["alt"]))
                                            idx += 1
                                        #Player A's ingame race played appears last
                                        else:
                                            PlayerARace.append(str(e["alt"]))

                            #Length of 5 indicates country is missing
                            elif(len(td) == 5):
                                idx = 0
                                #Insert blank string for missing name
                                PlayerACountry.append(" ")
                                for e in td:
                                    if(len(str(e)) > 1):
                                        #Player A's nmae appears first
                                        if(idx == 0):
                                            PlayerAName.append(str(e.contents[0]).strip())
                                            idx += 1
                                        #Player A's ingame race played appears last
                                        else:
                                            PlayerARace.append(str(e["alt"]))
                            #The winner of the Match was included as part of the td class name for both players.
                            #I just added the outcome based off of player A
                            if(td["class"][1] == "winner"):
                                PlayerAWin.append(1)
                            else:
                                PlayerAWin.append(0)


                        elif(td["class"][0] == "lm_score"):
                            MapScore.append(str(td.contents[0]).strip())

                        #Player B
                        elif(td["class"][0] == "lm_plb"):
                            #Some player information cells are missing the players country. len(td) of 7 indicates all fields present and 5 as missing the country info
                            if(len(td) == 7):
                                idx = 0
                                for e in td:
                                    #Ignore whitespace elements
                                    if(len(str(e)) > 1):
                                        #Player B's ingame race appears first
                                        if(idx == 0):
                                            PlayerBRace.append(str(e["alt"]))
                                            idx += 1
                                        #Player B's country appears next
                                        elif(idx == 1):
                                            PlayerBCountry.append(str(e["alt"]))
                                            idx += 1
                                        #Finally Player B's name
                                        else:
                                            PlayerBName.append(str(e.contents[0]).strip())
                            
                            #Length of 5 indicates country is missing
                            elif(len(td) == 5):
                                #Insert blank string for missing name
                                PlayerBCountry.append(" ")
                                idx = 0
                                for e in td:
                                    if(len(str(e)) > 1):
                                        if(idx == 0):
                                            PlayerBRace.append(str(e["alt"]))
                                            idx += 1
                                        else:
                                             PlayerBName.append(str(e.contents[0]).strip())
                        
                        elif(td["class"][0] == "lm_rtb"):
                            PlayerBRating.append(str(td.contents[0]).strip())

                        #This td contains information about the matches
                        elif(td["class"][0] == "lm_right"):
                            #Rated matches do not seem to be rated, I assumed that all matches not marked as unrated are rated.
                            if(len(td) == 5):
                                isRatedMatch.append("rated")
                                idx = 0
                                for e in td:
                                    if(len(str(e)) > 1):
                                        if(idx == 0):
                                            GameVersion.append(str(e["alt"]))
                                            idx += 1
                                        else:
                                            isLan.append(str(e["alt"]))
                            elif(len(td) == 7):
                                idx = 0
                                for e in td:
                                    if(len(str(e)) > 1):
                                        if(idx == 0):
                                            isRatedMatch.append(str(e["alt"]))
                                            idx += 1
                                        elif(idx == 1):
                                            GameVersion.append(str(e["alt"]))
                                            idx += 1
                                        else:
                                            isLan.append(str(e["alt"]))
                    except:
                        pass
    #Go to the next page
    driver.find_element(By.LINK_TEXT, "« day").click()

In [179]:
data = pd.DataFrame()
data["PlayerAName"] = PlayerAName
data["PlayerACountry"] = PlayerACountry
data["PlayerARating"] = PlayerARating
data["PlayerARace"] = PlayerARace
data["PlayerBName"] = PlayerBName
data["PlayerBCountry"] = PlayerBCountry
data["PlayerBRating"] = PlayerBRating
data["PlayerBRace"] = PlayerBRace
data["GameVersion"] = GameVersion
data["isRatedMatch"] = isRatedMatch
data["isLan"] = isLan
data["MapScore"] = MapScore
data["PlayerAWin"] = PlayerAWin

data.to_csv("outputDataMining.csv")